In [2]:
import requests
import pandas as pd


In [3]:
API_KEY = 'enjdK0BmrJmmdtDk8hm27eoUXv1lm9TfvDclcgW1'  # Replace with your actual API key

In [4]:
def search_food(query, page_size=5):
    """
    Search for food items matching the query.
    """
    url = 'https://api.nal.usda.gov/fdc/v1/foods/search'
    params = {
        'api_key': API_KEY,
        'query': query,
        'pageSize': page_size
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"API request failed with status code {response.status_code}")
    return response.json()


In [5]:
def get_food_details(fdc_id):
    """
    Retrieve detailed information for a specific food item by FDC ID.
    """
    url = f'https://api.nal.usda.gov/fdc/v1/food/{fdc_id}'
    params = {'api_key': API_KEY}
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"API request failed with status code {response.status_code}")
    return response.json()


In [6]:
def extract_nutrients(food_data):
    """
    Extract nutrient information from food data.
    """
    nutrients = food_data.get('foodNutrients', [])
    data = []
    for nutrient in nutrients:
        name = nutrient.get('nutrient', {}).get('name')
        amount = nutrient.get('amount')
        unit = nutrient.get('nutrient', {}).get('unitName')
        if name and amount is not None:
            data.append({'Nutrient': name, 'Amount': amount, 'Unit': unit})
    df = pd.DataFrame(data)
    return df


In [8]:
# Search for a food item
query = 'maggi noodles'
search_results = search_food(query)

# Display search results
for idx, food in enumerate(search_results.get('foods', []), start=1):
    description = food.get('description', 'No description')
    fdc_id = food.get('fdcId', 'No FDC ID')
    print(f"{idx}. {description} (FDC ID: {fdc_id})")

# Select the first result for demonstration
selected_food = search_results['foods'][0]
fdc_id = selected_food['fdcId']

# Retrieve detailed nutrient information
food_details = get_food_details(fdc_id)
print(food_details)
# Extract and display nutrients
nutrient_df = extract_nutrients(food_details)
print(nutrient_df)


1. Noodles, cooked (FDC ID: 2708352)
2. Adobo, with noodles (FDC ID: 2708809)
3. Noodle pudding (FDC ID: 2708949)
4. Noodles, chow mein (FDC ID: 2708354)
5. Beef and noodles, no sauce (FDC ID: 2706480)
{'foodClass': 'Survey', 'description': 'Noodles, cooked', 'foodNutrients': [{'type': 'FoodNutrient', 'id': 34329079, 'nutrient': {'id': 1003, 'number': '203', 'name': 'Protein', 'rank': 600, 'unitName': 'g'}, 'amount': 4.51}, {'type': 'FoodNutrient', 'id': 34329080, 'nutrient': {'id': 1004, 'number': '204', 'name': 'Total lipid (fat)', 'rank': 800, 'unitName': 'g'}, 'amount': 2.06}, {'type': 'FoodNutrient', 'id': 34329081, 'nutrient': {'id': 1005, 'number': '205', 'name': 'Carbohydrate, by difference', 'rank': 1110, 'unitName': 'g'}, 'amount': 25.0}, {'type': 'FoodNutrient', 'id': 34329082, 'nutrient': {'id': 1008, 'number': '208', 'name': 'Energy', 'rank': 300, 'unitName': 'kcal'}, 'amount': 137}, {'type': 'FoodNutrient', 'id': 34329083, 'nutrient': {'id': 1018, 'number': '221', 'name':

In [1]:
import json

# Load the uploaded JSON data
with open('../data/FoodData_Central_branded_food_json_2025-04-24.json', 'r') as file:
    data = json.load(file)

# Function to search for nutrition values based on food description
def get_nutrition_for_meal(meal_name):
    # Normalize the meal name to lowercase for case-insensitive comparison
    meal_name = meal_name.lower()
    
    # Iterate through all foods in the dataset
    for food in data['FoundationFoods']:
        # Compare description with the given meal name
        if meal_name in food['description'].lower():
            # If match is found, extract relevant nutritional information
            nutrition_info = []
            serving_size = None

            # Extract serving size if available
            if food.get('foodPortions'):
                serving_size = food['foodPortions'][0].get('value', 'N/A')
                serving_unit = food['foodPortions'][0].get('measureUnit', {}).get('abbreviation', '')
                serving_size = f"{serving_size} {serving_unit}" if serving_size != 'N/A' else 'N/A'

            # Add nutrients data
            for nutrient in food['foodNutrients']:
                nutrient_data = {
                    "name": nutrient['nutrient']['name'],
                    "amount": nutrient['amount'],
                    "unit": nutrient['nutrient']['unitName']
                }
                nutrition_info.append(nutrient_data)
            
            return nutrition_info, serving_size
    
    # Return a message if no matching meal is found
    return f"No nutritional information found for '{meal_name}'.", None

# Example: Get nutrition for "Butter Chicken"
meal_name = "Butter Chicken"
nutrition_values, serving_size = get_nutrition_for_meal(meal_name)

# Check if the result is a list of nutrients or an error message
if isinstance(nutrition_values, list):
    # Print the serving size
    print(f"Serving Size: {serving_size}")
    
    # Display the nutritional information
    for nutrient in nutrition_values:
        print(f"{nutrient['name']}: {nutrient['amount']} {nutrient['unit']}")
else:
    # If no match was found, print the error message
    print(nutrition_values)


: 